# Molecular Simulation

Using Lennard-Jones potential

In [2]:
import numpy as np
import pygame
from IPython.display import display, clear_output

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
N_PARTICLES = 2
MASS = 1.
SIGMA = 1.
EPSILON = 1.

In [22]:
# Force derived from Lennard-Jones potential, F = -gradV
def lennard_jones_force(X):
	'''
	X = [dpx, dpy, vx, vy], an ndarray
	dpx, dpy: difference between position of molecule that
	will be updated and molecule that is creating the force
	vx, vy: velocity of molecule that will be updated
	'''
	dpx, dpy, vx, vy = X
	Dpx = vx
	Dpy = vy
	drs = dpx**2 + dpy**2 + 1e0 # Distance between particles squared
	c = 24 * EPSILON * (SIGMA ** 6) * (1 / MASS) * (drs ** (-4)) * (2 * (SIGMA ** 6) * (drs ** (-3)) - 1) # Insane constant
	Dvx = c * dpx
	Dvy = c * dpy
	return np.array([Dpx, Dpy, Dvx, Dvy])

In [23]:
def euler(F, X, h):
	X_next = X + h * F(X)
	return X_next

In [24]:
# Update particles according to the ODE that determines the system
def simulation_step(particles, h):
	updated_particles = np.zeros_like(particles)
	for i, p1 in enumerate(particles):
		for p2 in particles:
			if p1 is not p2:
				dpx, dpy = p1[:2] - p2[:2]
				vx, vy = p1[2:]
				X = np.array([dpx, dpy, vx, vy])
				# X = np.concatenate((p1[:2] - p2[:2]), p1[2:]) FIX THIS LATER I GUESS
				updated_particles[i] += euler(lennard_jones_force, X, h)
	return updated_particles

In [25]:
# Each particle has an px, py position and vx, vy velocity
particles = np.random.uniform(size=(N_PARTICLES, 4))
print(particles[:, :2])

[[0.89718669 0.61381532]
 [0.09782131 0.79342911]]


In [26]:
for _ in range(100):
	particles = simulation_step(particles, 1e-10)
	print(particles[0, :2])

[ 0.79936538 -0.17961379]
[ 1.59873076 -0.35922758]
[ 3.19746152 -0.71845515]
[ 6.39492304 -1.4369103 ]
[12.78984608 -2.8738206 ]
[25.57969216 -5.74764121]
[ 51.15938431 -11.49528241]
[102.31876863 -22.99056481]
[204.63753725 -45.9811296 ]
[409.27507451 -91.96225917]
[ 818.55014902 -183.92451827]
[1637.10029803 -367.84903642]
[3274.20059606 -735.6980726 ]
[ 6548.40119212 -1471.39614471]
[13096.80238422 -2942.79228843]
[26193.60476842 -5885.58457488]
[ 52387.20953677 -11771.16914581]
[104774.41907342 -23542.33828374]
[209548.8381466  -47084.67655169]
[419097.67629272 -94169.35307181]
[ 838195.35258445 -188338.7060805 ]
[1676390.70516695 -376677.41203474]
[3352781.41032999 -753354.82381697]
[ 6705562.82065216 -1506709.64712892]
[13411125.64128869 -3013419.29324779]
[26822251.2825461  -6026838.58447547]
[ 53644502.56502964 -12053677.16491073]
[ 1.07289005e+08 -2.41073543e+07]
[ 2.14578010e+08 -4.82147086e+07]
[ 4.29156021e+08 -9.64294172e+07]
[ 8.58312041e+08 -1.92858834e+08]
[ 1.71662408

In [20]:
# Initialize pygame display
def init_pygame_display(window_size=(500, 500)):
    pygame.init()
    screen = pygame.display.set_mode(window_size)
    pygame.display.set_caption("Position Simulation")
    return screen

# Function to display simulation using pygame
def run_simulation(particles, window_size=(500, 500), point_radius=5, speed=0.1):
    screen = init_pygame_display(window_size)
    clock = pygame.time.Clock()
    running = True
    
    while running:
        # Update particles
        particles = simulation_step(particles, 1e-10)
        
		# Pygame stuff
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()

        screen.fill((255, 255, 255))  # Clear screen with white background

        # Draw each point from positions array
        for pos in particles:
            print(pos)
            x, y = pos[0], pos[1]
            pygame.draw.circle(screen, (0, 0, 255), (int(x), int(y)), point_radius)

        pygame.display.flip()  # Update the display
        clock.tick(30)  # Cap the frame rate at 30 FPS

In [21]:
#run_simulation(particles)